## Setup

In [1]:
import os
import sys
import pandas as pd
import boto3
import numpy as np
from sklearn.preprocessing import StandardScaler
from io import StringIO
from dotenv import load_dotenv

sys.path.append("src")
from mlops_project.utils.load_from_s3 import S3Loader


In [2]:
load_dotenv()

bucket = os.getenv("S3_BUCKET_NAME")
filename = os.getenv("CSV_FILENAME")
s3_key = f"datasets/{filename}"

target = "median_house_value"
id_column = None # None if no index value

s3loader = S3Loader(bucket)
df = s3loader.load_csv_from_s3(s3_key)
print(f"✅ Loaded dataset with shape: {df.shape}")

🌀 GZIP compression detected
✅ Loaded dataset with shape: (20640, 10)


## Initial Cleanup

In [3]:
# Initial dataset shape
print(f"🔍 Initial shape: {df.shape}")

🔍 Initial shape: (20640, 10)


In [4]:
# Drop duplicate rows
df = df.drop_duplicates()
print(f"🧹 After removing duplicates: {df.shape}")

🧹 After removing duplicates: (20640, 10)


In [5]:
# Drop empty columns (100% missing values)
empty_cols = df.columns[df.isna().sum() == len(df)]
df = df.drop(columns=empty_cols)
print(f"🗑️ Dropped empty columns: {list(empty_cols)}")

🗑️ Dropped empty columns: []


In [6]:
# Drop constant columns (only one unique value)
constant_cols = [col for col in df.columns if df[col].nunique() <= 1]
df = df.drop(columns=constant_cols)
print(f"🧺 Dropped constant columns: {constant_cols}")

🧺 Dropped constant columns: []


In [7]:
# Set ID column as index (if defined, unique, and non-null)
if id_column and id_column in df.columns:
    is_unique = df[id_column].is_unique
    has_no_nan = df[id_column].isna().sum() == 0

    if is_unique and has_no_nan:
        df.set_index(id_column, inplace=True)
        print(f"📎 Set '{id_column}' as index.")
    elif not is_unique:
        print(f"⚠️ ID column '{id_column}' is not unique – not set as index.")
    elif not has_no_nan:
        print(f"⚠️ ID column '{id_column}' contains missing values – not set as index.")

In [8]:
# Drop unique columns (as many unique values as rows)
unique_cols = [col for col in df.columns if df[col].nunique() == len(df)]
df = df.drop(columns=unique_cols)
print(f"🚮 Dropped totally unique columns: {unique_cols}")

🚮 Dropped totally unique columns: []


In [9]:
print(f"✅ Cleaned shape: {df.shape}")

✅ Cleaned shape: (20640, 10)


## Handling Missing Values (numerical features)

In [10]:
# Select numerical columns
num_cols = df.select_dtypes(include=["number"]).columns

In [11]:
# Count missing values in numerical columns
missing_num = df[num_cols].isna().sum()
missing_num = missing_num[missing_num > 0]

print("🔍 Numerical columns with missing values:")
print(missing_num)

🔍 Numerical columns with missing values:
total_bedrooms    207
dtype: int64


In [12]:
# Fill missing values with the median of each column
for col in missing_num.index:
    median_value = df[col].median()
    df[col] = df[col].fillna(median_value)
    print(f"🧪 Filled NaNs in '{col}' with median: {median_value:.2f}")

🧪 Filled NaNs in 'total_bedrooms' with median: 435.00


## Handling Missing Values (categorical features)

In [13]:
# Select categorical columns
cat_cols = df.select_dtypes(include=["object", "category"]).columns

In [14]:
# Count missing values in categorical columns
missing_cat = df[cat_cols].isna().sum()
missing_cat = missing_cat[missing_cat > 0]

print("🔍 Categorical columns with missing values:")
print(missing_cat)

🔍 Categorical columns with missing values:
Series([], dtype: int64)


In [15]:
# Fill missing values with the most frequent value (mode)
for col in missing_cat.index:
    most_common = df[col].mode()[0]
    df[col] = df[col].fillna(most_common)
    print(f"📌 Filled NaNs in '{col}' with most frequent value: '{most_common}'")

## Standardisation

In [16]:
# Select numerical columns
num_cols = df.select_dtypes(include=["number"]).columns
num_cols = [col for col in num_cols if col != target]

In [17]:
# Identify numeric columns with few unique values (discrete-like)
discrete_as_cat = [col for col in num_cols if df[col].nunique() <= 5]

In [18]:
# Final list of columns to scale = numeric columns - discrete ones
scale_cols = [col for col in num_cols if col not in discrete_as_cat]

In [19]:
print(f"🧠 Treating as categorical (discrete numeric): {discrete_as_cat}")
print(f"📏 Standardizing columns: {scale_cols}")

🧠 Treating as categorical (discrete numeric): []
📏 Standardizing columns: ['longitude', 'latitude', 'housing_median_age', 'total_rooms', 'total_bedrooms', 'population', 'households', 'median_income']


In [20]:
# Apply StandardScaler only on selected columns
scaler = StandardScaler()
scaled_data = scaler.fit_transform(df[scale_cols])

In [21]:
# Replace in DataFrame
df[scale_cols] = scaled_data

In [22]:
print(f"✅ Standardized {len(scale_cols)} columns.")

✅ Standardized 8 columns.


## Encoding

In [23]:
# Get original categorical columns (object or category type)
base_cat_cols = df.select_dtypes(include=["object", "category"]).columns.tolist()
base_cat_cols = [col for col in base_cat_cols if col != target]

In [24]:
# Add discrete numeric columns identified as categorical
cat_cols = base_cat_cols + discrete_as_cat

In [25]:
print(f"🎯 One-hot encoding on columns: {cat_cols}")

🎯 One-hot encoding on columns: ['ocean_proximity']


In [26]:
df_encoded = pd.get_dummies(df, columns=cat_cols)

In [27]:
print(f"✅ Shape after encoding: {df_encoded.shape}")

✅ Shape after encoding: (20640, 14)


In [28]:
df_encoded

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity_<1H OCEAN,ocean_proximity_INLAND,ocean_proximity_ISLAND,ocean_proximity_NEAR BAY,ocean_proximity_NEAR OCEAN
0,-1.327835,1.052548,0.982143,-0.804819,-0.972476,-0.974429,-0.977033,2.344766,452600.0,False,False,False,True,False
1,-1.322844,1.043185,-0.607019,2.045890,1.357143,0.861439,1.669961,2.332238,358500.0,False,False,False,True,False
2,-1.332827,1.038503,1.856182,-0.535746,-0.827024,-0.820777,-0.843637,1.782699,352100.0,False,False,False,True,False
3,-1.337818,1.038503,1.856182,-0.624215,-0.719723,-0.766028,-0.733781,0.932968,341300.0,False,False,False,True,False
4,-1.337818,1.038503,1.856182,-0.462404,-0.612423,-0.759847,-0.629157,-0.012881,342200.0,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20635,-0.758826,1.801647,-0.289187,-0.444985,-0.388283,-0.512592,-0.443449,-1.216128,78100.0,False,True,False,False,False
20636,-0.818722,1.806329,-0.845393,-0.888704,-0.922403,-0.944405,-1.008420,-0.691593,77100.0,False,True,False,False,False
20637,-0.823713,1.778237,-0.924851,-0.174995,-0.123608,-0.369537,-0.174042,-1.142593,92300.0,False,True,False,False,False
20638,-0.873626,1.778237,-0.845393,-0.355600,-0.304827,-0.604429,-0.393753,-1.054583,84700.0,False,True,False,False,False


## Export

In [29]:
s3 = boto3.client("s3")
output_key = f"datasets/{filename.replace('.csv', '_processed.csv')}"

In [30]:
csv_buffer = StringIO()
df_encoded.to_csv(csv_buffer, index=True)

In [31]:
# Upload to S3
s3.put_object(
    Bucket=os.getenv("S3_BUCKET_NAME"),
    Key=output_key,
    Body=csv_buffer.getvalue()
)

print(f"✅ Saved processed dataset to s3://{os.getenv('S3_BUCKET_NAME')}/{output_key}")

✅ Saved processed dataset to s3://athos-mlops0-bucket/datasets/california_housing_processed.csv
